In [ ]:
import itertools
import random

In [ ]:
K = (0x19B2, 0xCD91, 0x8919, 0x02199F9A)

In [ ]:
for i in [3, 4, 5]:
    j = i>>1
    print(i,j, bin(i), bin(j))
    j = i<<1
    print(i,j, bin(i), bin(j))


In [ ]:
print(0xFFFFFFFFFFFFFFFF)
print(2**64-1)
print(len("FFFFFFFFFFFFFFFF"))
print(2**(4*16))

In [ ]:
ONE_WORD = 8

def rotate_left(x, d):
    """Rotate x by d bits to the left."""
    d = d % W_L
    out = x << d
    excess = out 
    out = out & (2**W_L-1)
    for i in range(d):
        bit = (x & 2**(W_L-1-d+1+i))>> (W_L-1-d+1+i)
        out |= bit << i
    return out

def rotate_left_2(x, d):
    return ((x << d) | (x >> (ONE_WORD-d))) & 2**ONE_WORD-1


def rotl_64(x, d):
    return ((x << d) | (x >> (64-d))) & 2**64-1

MASK = 2**64-1

def mix(x0, x1, R):
    y0 = (x0+x1) & MASK
    y1 = rotl_64(x1, R) ^ y0
    return y0, y1

K = (0x19B2, 0xCD91, 0x8919, 0x02199F9A)
K = (K[0], K[1], K[2], K[3], C240^K[0]^K[1]^K[2]^K[3])
TW = (192, 810)
TW = (TW[0], TW[1], TW[0]^TW[1])
def key_schedule(s):
    return (K[(s)%(NW+1)] & MASK,
              (K[(s+1)%(NW+1)] + TW[s%3]) & MASK,
              (K[(s+2)%(NW+1)] + TW[(s+1)%3]) & MASK,
              (K[(s+3)%(NW+1)] + s) & MASK)
    

In [ ]:
tf_rot = (16, 42, 12, 31, 16, 32, 24, 21)
tf_rot = itertools.cycle(tf_rot)

In [ ]:
NW = 4
C240 = 0x1BD11BDAA9FC1A22
N_ROUNDS=72
MASK = 2**64-1
pi = (0, 3, 2, 1)
R_256 = ((14, 16), (52, 57), (23, 40), (5, 37), (25, 33), (46, 12), (58, 22), (32, 32))

def mix(x0, x1, R):
    y0 = (x0+x1) & MASK
    y1 = rotl_64(x1, R) ^ y0
    return y0, y1

def key_schedule(K, s):
    return (K[(s)%(NW+1)] & MASK,
              (K[(s+1)%(NW+1)] + TW[s%3]) & MASK,
              (K[(s+2)%(NW+1)] + TW[(s+1)%3]) & MASK,
              (K[(s+3)%(NW+1)] + s) & MASK)

def threefish(p, K, TW):
    v = list(p)
    for r in range(N_ROUNDS):
        e = [0]*4
        if r%4 == 0:
            ksi = key_schedule(K, r//4)
            for i in range(NW):
                e[i] = (v[i] + ksi[i]) & MASK
            print('key injection   ', list(map(hex, e)))
        else:
            e = v
        f = [0]*4
        f[0], f[1] = mix(e[0], e[1], R_256[r%8][0])
        f[2], f[3] = mix(e[2], e[3], R_256[r%8][1])
        if r%2 == 0: print('end of round %03i' % (r+1), list(map(hex, f)))
        for i in range(NW):
            v[i] = f[pi[i]]
        if r%2 == 1: print('end of round %03i' % (r+1), list(map(hex, v)))

    ksi = key_schedule(K, N_ROUNDS//4)
    v = [((x + k)^pp) & MASK for x, k, pp in zip(v, ksi, p)]
    print(list(map(hex, v)))

    return v


In [ ]:
K = (0x0, 0x0, 0x0, 0x0)
K = (K[0], K[1], K[2], K[3], C240^K[0]^K[1]^K[2]^K[3])
TW = (0x0, 0x0)
TW = (TW[0], TW[1], TW[0]^TW[1])

c = threefish([0x0]*4, K, TW)


In [ ]:
print(list(map(hex, c)))
print(list(map(hex, K)))

In [ ]:
K = (0x1716151413121110, 0x1F1E1D1C1B1A1918, 0x2726252423222120, 0x2F2E2D2C2B2A2928)
TW = (0x0706050403020100, 0x0F0E0D0C0B0A0908)
K = (K[0], K[1], K[2], K[3], C240^K[0]^K[1]^K[2]^K[3])
TW = (TW[0], TW[1], TW[0]^TW[1])

c = threefish((0xF8F9FAFBFCFDFEFF, 0xF0F1F2F3F4F5F6F7, 0xE8E9EAEBECEDEEEF, 0xE0E1E2E3E4E5E6E7), K, TW)


In [ ]:
print(type(1), type(2**63), type(2**64), type(2**64-1))

In [ ]:
K